# Introduction à Spark SQL

Spark SQL est un module d'Apache Spark, qui facilite la mise en place de traitement sur des données à haute volumétrie :
 * **structurées** : la donnée est stockée sous un format standardisé (CSV, JSON, Avro, Parquet...) et répond à une structure partagée (ie. schéma) répondant à un besoin technique ou métier
 * **semi-structurées** : la donnée est stockée sous un format standardisé, mais sa structure interne n'est pas connue par avance.

Spark SQL offre une interface pour interagir avec les données via le langage SQL, ainsi que des fonctionnalités pour la lecture et l'écriture de données dans divers formats. Spark SQL facilite l'intégration entre le traitement des données relationnelles et le traitement distribué à grande échelle en utilisant les DataFrames et les Datasets, deux structures de données immuables.

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.4.1`
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .progress(enable = true, keep = true, useBars = false)
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("SparkSQL - Introduction")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit des fonctions supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper
import spark_helper.implicits._

**Note** : la variable `spark` définie ci-dessus représente la session SparkSQL.

La ligne `import spark.implicits._` permet de récupérer des codecs permettant de gérer les données sérialisées, ainsi que la possibilité d'utiliser la notation `$"<nom-colonne>"` pour référencer des colonnes.

## Lecture d'un fichier avec Spark SQL

Nous allons récupérer le fichier `orders.csv` et réaliser des analyses sur ce fichier.

Commençons par afficher un extrait de son contenu.

In [ ]:
%%shell

cat data/orders.csv

### Lecture : première approche
La récupération du contenu d'un fichier avec Spark SQL va s'avérer beaucoup plus simple qu'avec Spark Core, car Spark SQL est fourni avec un ensemble de codec pour gérer les formats CSV, JSON, texte, binaire, Avro, Parquet, ORC.

Nous allons récupérer le contenu du fichier `data/orders.csv`. Dans le cadre de SparkSQL, la récupération d'un fichier commence par l'appel à `spark.read`, chaîner éventuellement avec une succession d'option (ie. `.option("<option_name>", <option_value>)`). Le chaînage se termine par l'appel d'une méthode dont le nom représente le format de donnée.

Ci-dessous nous utilisons l'options `header`, qui permet d'indiquer que la première du fichier contient les en-têtes de colonne. Ces en-têtes sont ensuite utilisées pour nommer les colonnes parmis les données récupérées par SparkSQL.
 
D'autres options sont disponibles pour, par exemple, préciser un séparateur de colonne différent, le format de date utilisé, l'utilisation d'un algorithme de compression (`none`, `bzip2`, `gzip`, `lz4`, `snappy`, `deflate`)... Consultez la [documentation à ce sujet](https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option) pour la liste exhaustive.

#### Dataframe

Le résultat de ce chaînage d'appel est un dataframe. Un dataframe, comme les RDD, est une abstraction de données distribuées dans Apache Spark. Contrairement aux RDD, les dataframes sont spécifiquement conçus pour faciliter le traitement et l'analyse de données structurées et semi-structurées. Ils sont inspirés des DataFrames de R et de la bibliothèque Pandas du langage Python. Cette abstraction offre une API haut niveau pour travailler avec des données tabulaires dans un contexte distribué, en se basant sur des opérations propres au langage SQL.

Note : la méthode `.showHTML()` et le _magic hook_ `%%data` permettent aussi d'afficher un extrait d'un dataframe. Normalement, pour afficher un dataframe, il faut appeler dessus la méthode `.show()`, fournie par Spark SQL.

In [ ]:
%%data limit=10

val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // lecture du fichier au format CSV
    .csv("data/orders.csv")

dataframe

Juste à titre d'exemple, voici ce que donne comme affichage la méthode `.show()`

In [ ]:
dataframe.show(10)

Nous allons afficher le schéma associé, afin de comprendre ce qui a été récupérer.

In [ ]:
dataframe.printSchema()

**Ce qu'il faut voir**

Tout d'abord, pour récupérer le contenu d'un fichier SparkSQL a exécuté 2 jobs, que vous pouvez voir dans Spark UI. Le premier job (csv) a permis de récupérer les en-têtes de colonne. Le second job (showHTML) a permis de récupérer juste les lignes nécessaires pour l'affichage.

Nous pouvons voir un nouvel onglet dans l'affichage de Spark UI. Il s'agit de "SQL / DataFrame". Celui-ci permet de voir l'ensemble des requêtes SparkSQL que vous avez exécutés, avec leur job associé, leur plan d'exécution sous forme de DAG et comme le nombre de lignes récupérer, le nombre de fichiers traités.

Nous voyons que lorsque SparkSQL récupère des données SQL, par défaut toutes les données sont interprétées comme des chaînes de caractères. Sachant que nous avons des dates et des prix. Ceci ne nous convient pas. Nous allons voir si nous pouvons faire mieux.

### Lecture : deuxième approche

Nous allons utiliser une autre options
 * `inferSchema` (`true`/`false`) : demande à SparkSQL de réaliser une pré-analyse des données du fichier pour déterminer le type associé à chaque colonne.

Utilisez cette option dans le code ci-dessous.

In [ ]:
%%data limit=10

val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // lecture du fichier au format CSV
    .csv("data/orders.csv")

dataframe

**Ce qu'il faut voir**

Dans la Spark UI, vous pouvez voir un nouvel onglet dans la barre du haut intitulé "SQL / DataFrame". En cliquant dessus, vous verrez apparaître les requêtes exécutées par Spark SQL. Si vous cliquez sur une requête, vous verrez un diagramme représentant le plan d'exécution et dans la partie "Details" une représentation textuelle du plan d'exécution.

Affichons le schéma de notre dataframe.

In [ ]:
dataframe.printSchema()

**Ce qu'il faut voir**

Avec l'instruction `dataframe.printSchema`, nous pouvons voir que Spark a réussi à déterminer le schéma des données du fichier. Ce qui inclut le fait de déterminer le nom des colonnes et de déterminer le type des colonnes (grâce à l'option `inferSchema` pour ce dernier). Cependant, l'option `inferSchema` a deux problèmes :

 * Il nécessite une lecture supplémentaire du fichier (sur un extrait). Si vous regardez les barres de progression ci-dessus et dans Spark UI, vous verrez deux étapes de lecture CSV.
 * Il peut se tromper.

### Lecture : troisième approche
Nous allons maintenant relire le fichier CSV, mais cette fois en fournissant directement un schéma entré à la main.

Cette fois, vous n'utiliserez pas l'option `inferSchema`. À la place, vous utiliserez la méthode `.schema()`, avant d'appeler `.csv()`, avec le schéma suivant à passer en paramètre :

```
"id STRING, client STRING, timestamp TIMESTAMP, product STRING, price DOUBLE"
```

In [ ]:
%%data limit=10

val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // lecture du fichier au format CSV
    .csv("data/orders.csv")

dataframe

**Ce qu'il faut voir**

Le fait de fournir un schéma va inciter Spark SQL à ne pas réaliser des analyses préalables ou des vérifications. Nous voyons, en effet, que l'ensemble du process est réduit à un job au lieu de deux ou trois.

## Dataset

Dataframe est une abstraction générique. Il y a certes un schéma associé aux données récupérées, mais les données du dataframe n'est pas associé à un modèle mémoire.

Dans le cadre des langages Scala et Java, SparkSQL fournit une autre abstraction : `Dataset[A]`. Cette abstraction permet d'associer des données récupérées avec SparkSQL à un modèle défini en mémoire : typiquement une classe Java ou une case class Scala.

Un `Dataset` peut se créer à partir d'une collection ou d'un RDD. Il est possible d'en créer depuis un `DataFrame` en utilisant la méthode `.as[T]`, où `T` représente le type (case classe) que doit le `Dataset`. Veillez à ce que le nom des champs de votre case class corresponde correctement au nom des colonnes du `DataFrame`. N'hésitez pas à renommer les colonnes, si besoin.

**Note** : en réalité, un `DataFrame` est alias du type `Dataset[Row]`, où `Row` est une représentation générique d'une ligne de données.

Ci-dessous, créez un `Dataset` à partir du dataframe défini plus haut et de la case class Order. N'hésitez pas à utiliser la méthode `.withColumnRenamed(<ancient>, <nouveau>)` pour renommer des colonnes si nécessaire.

In [ ]:
%%data limit=10

import java.sql.Timestamp

case class Order(
  id:        String,
  clientId:  String,
  timestamp: Timestamp,
  product:   String,
  price:     Double
)

// Cette est nécessaire ici uniquement, dans le cadre de ce notebook.
// Normalement, vous n'avez pas besoins d'y faire appel.
spark_helper.sparkExport(this)

val orders: Dataset[Order] =
  dataframe
    .withColumnRenamed("client", "clientId")
    .as[Order]

orders

## Trouvez le produit le plus vendu (ID du produit et quantité totale vendue)

### Première approche : utilisation de l'API SparkSQL

Lorsque vous utilisez l'API SparkSQL, vous allez utiliser des méthodes comme `.select()`, `.where()`, `.groupBy()`... Ces méthodes prennent en paramètre des références sur des colonnes.

Il y a trois manières de référencer une colonne :
 * `dataframe("nom-colonne")` : en utilisant un dataframe/dataset défini
 * `col("nom-colonne")` : en utilisant les fonctions prédéfinies dans `org.apache.spark.sql.functions._`
 * `$"nom-colonne"` (Scala uniquement) : si vous avez importé `spark.implicits._`, où `spark` est la session Spark.
 
Il existe des références spéciales à des colonnes, autrement dit, des colonnes qui n'en sont pas toujours :
 * `lit(constante)` : permet d'utiliser une constante au niveau d'une colonne.
 * les fonctions provenant de `org.apache.spark.sql.functions._` fournissent en sortie une référence spéciale à une colonne indépendamment de leur finalité.
 * `*` : référence toutes les colonnes d'un dataset.
 * `a.b` : permet d'extraire le champ `b` d'une colonne `a`, lorsque `a` contient une sous-structure.
 * `a.*` : permet d'extraire tous les champs d'une colonne `a`, lorsque `a` contient une sous-structure.
 * `a[n]` : permet d'extraire l'élément d'index `n` de la colonne `a`, lorsque `a` contient une liste.
 
L'ensemble des méthodes applicables sont disponibles dans la [Scaladoc de SparkSQL](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/Dataset.html).

In [ ]:
// utilisez .groupBy(colonne) pour regroupes les commandes par produit
val ordersByProduct = ???

// pas d'affichage possible ici :/

Maintenant que nous avons regroupé les entités par clé (ie. le même produit), nous allons agréger ces entités.

L'agrégation se fait avec la méthode `.agg()`. Cette fonction prend en paramètre une fonction d'agrégation.

Par exemple pour compter le nombre d'instances d'une clé :

```scala
df.groupBy($"keyCol").agg(count(lit(1)).as("count"))
```

Dans ce code, `.as()` permet de donner un nom expoitable à une colonne. Sans cet alias, vous allez vous retrouver avec une colonne qui s'appelle `count(1)`

L'ensemble des fonctions applicables dans la méthode `.agg()` sont disponibles dans la section "Aggregate functions" de la [Scaladoc de functions](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/functions$.html).

In [ ]:
%%data

// utilisez .agg() et la fonction count() pour compter le nombre de commandes par produit
val countOrdersByProduct = ???

countOrdersByProduct

Nous allons maintenant trier les scores des produits, en commençant par le produit le plus vendu. Pour cela, nous allons utiliser la méthode `.orderBy()`, qui permet de trier par rapport à une colonne.

Il est possible d'utiliser la méthode `.desc` sur une colonne pour indiquer qu'on souhaite que le tri est décroisant.

In [ ]:
%%data

// utilisez .orderBy(colonne) pour trier les produits
val sortedCountOrdersByProduct = ???

sortedCountOrdersByProduct

### Deuxième approche : utilisation d'une requête SQL

Nous allons faire le même exercice, mais cette fois en utilisant une requête SQL.

Pour cela nous devons créer une vue (SQL) sur notre dataset.

In [ ]:
orders.createOrReplaceTempView("orders")

Nous pouvons maintenant écrire la requête SQL en utilisant la vue crée.

Complétez la requête ci-dessous, en vous assurant que vous obtenez bien le même résultat que dans le cadre de l'exercice précédent.

In [ ]:
spark.sql("""
SELECT ???
FROM ???
GROUP BY ???
ORDER BY ??? DESC
""").showHTML()

Avec les facilités offertes dans ces notebooks, vous pouvez écrire directement la requête de cette façon.

In [ ]:
%%sql

SELECT ???
FROM ???
GROUP BY ???
ORDER BY ??? DESC

**Ce qu'il faut voir**

Si vous regardez le plan d'exécution de cette requête et que vous le comparez au dernier plan d'exécution obtenu à travers l'utilisation de l'API Spark SQL, vous remarquerez que ces deux plans d'exécution sont identiques. Ce qui indique bien que les deux approches font exactement la même chose et qu'elles le font avec les mêmes performances.

Ainsi, Spark SQL vous donne la possibilité d'utiliser le langage qui vous convient le plus, tout en ayant le même comportement de la part de Spark. Ceci est vrai dans la majorité des cas, si vous vous tenez aux fonctions de base fournies par Spark SQL. C'est moins vrai dès que vous introduisez des éléments personnalisés (eg. UDF).